# TensorFlow Neural Network for XOR

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
xy = np.loadtxt('train_nn_xor.txt', unpack=False)
xy

array([[ 0.,  0.,  0.],
       [ 0.,  1.,  1.],
       [ 1.,  0.,  1.],
       [ 1.,  1.,  0.]])

In [3]:
x_data = xy[:, 0:-1]
x_data

array([[ 0.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  1.]])

In [4]:
print(len(x_data))
print(len(x_data[0]))

4
2


In [5]:
y_data = np.reshape(xy[:, -1], (4,1))
y_data

array([[ 0.],
       [ 1.],
       [ 1.],
       [ 0.]])

In [6]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

## 먼저 Logistic Regression  으로 작업해 본다면

In [7]:
W = tf.Variable(tf.random_uniform([len(x_data[0]), 1], -1.0, 1.0))

In [8]:
h = tf.matmul(X, W)

In [9]:
hypo = tf.div(1. , 1. + tf.exp(-h))
# hypo = tf.sigmoid(h) 를 풀어쓰면 위와 같습니다.

In [10]:
cost = -tf.reduce_mean(Y*tf.log(hypo) + (1-Y)*tf.log(1-hypo))

In [11]:
alpha = tf.Variable(0.1)
optimizer = tf.train.GradientDescentOptimizer(alpha)
train = optimizer.minimize(cost)

In [12]:
init = tf.initialize_all_variables()

In [13]:
sess = tf.Session()
sess.run(init)

In [14]:
for step in range(1000):
    sess.run(train, feed_dict={X:x_data, Y:y_data})
    if step % 200 == 0:
        print("step : ", step)
        print("cost : ", sess.run(cost, feed_dict={X:x_data, Y:y_data}))
        print(sess.run(W))
        print("="*50)


step :  0
cost :  0.731819
[[-0.72030699]
 [-0.13553387]]
step :  200
cost :  0.693611
[[-0.09440705]
 [ 0.07432526]]
step :  400
cost :  0.693183
[[-0.02431148]
 [ 0.02385534]]
step :  600
cost :  0.69315
[[-0.00687833]
 [ 0.00686797]]
step :  800
cost :  0.693147
[[-0.00196162]
 [ 0.00196138]]


In [15]:
corrent_prediction = tf.equal(tf.floor(hypo + 0.5), Y)

In [16]:
accuracy = tf.reduce_mean(tf.cast(corrent_prediction, 'float'))

In [17]:
print("hypo : \n", sess.run(hypo, feed_dict={X:x_data, Y:y_data}))
print("floor : \n", sess.run(tf.floor(hypo+0.5), feed_dict={X:x_data, Y:y_data}))
print("corrent_prediction : \n", sess.run(corrent_prediction, feed_dict={X:x_data, Y:y_data}))
print("accuracy : \n", sess.run(accuracy, feed_dict={X:x_data, Y:y_data}))

hypo : 
 [[ 0.5       ]
 [ 0.50014079]
 [ 0.49985912]
 [ 0.5       ]]
floor : 
 [[ 1.]
 [ 1.]
 [ 0.]
 [ 1.]]
corrent_prediction : 
 [[False]
 [ True]
 [False]
 [False]]
accuracy : 
 0.25


## 결론 : Logistic Regression 으로는 이론적으로도 실질적으로도 0.5 이상 나올 수가 없다.

## 그렇다면, Layer 를 한개 더 둠으로써, NN 을 이용해서 구해보자

In [18]:
W1 = tf.Variable(tf.random_uniform([2,2], -1.0, 1.0))
W2 = tf.Variable(tf.random_uniform([2,1], -1.0, 1.0))

In [19]:
b1 = tf.Variable(tf.zeros([2]), name="Bias1")
b2 = tf.Variable(tf.zeros([1]), name="Bias2")

In [20]:
# Define Layer2
L2 = tf.sigmoid(tf.matmul(X, W1) + b1)

In [21]:
# Define Hypo Function
hypo = tf.sigmoid(tf.matmul(L2, W2) + b2)

In [22]:
# Define Cost Function
cost = -tf.reduce_mean(Y*tf.log(hypo) + (1-Y)*tf.log(1. - hypo))

In [23]:
# Minimize
alpha = tf.Variable(0.1)
optimizer = tf.train.GradientDescentOptimizer(alpha)
train = optimizer.minimize(cost)

In [40]:
init = tf.initialize_all_variables()

In [41]:
sess = tf.Session()
sess.run(init)

In [42]:
for step in range(10000):
    sess.run(train, feed_dict={X:x_data, Y:y_data})
    if step % 2000 == 0:
        print("step : ", step)
        print("cost : ", sess.run(cost, feed_dict={X:x_data, Y:y_data}))
        print("W1 : \n", sess.run(W1, feed_dict={X:x_data, Y:y_data}))
        print("W2 : \n", sess.run(W2, feed_dict={X:x_data, Y:y_data}))
        print("="*60)


step :  0
cost :  0.840066
W1 : 
 [[ 0.3595995  -0.13043241 -0.1837099  -0.23773938  0.00861581  0.57257885
   0.77715933  0.05348545  0.89103347  0.96544778]
 [ 0.85606402  0.79053497  0.55438542 -0.8337965   0.04768913 -0.57149667
  -0.50974476  0.99481124  0.89738971  0.04536261]]
W2 : 
 [[-0.46030766]
 [-0.96081531]
 [ 0.60846549]
 [-0.68036109]
 [-0.59672469]
 [ 0.62184668]
 [-0.36329108]
 [-0.56613868]
 [-0.04485464]
 [ 0.33153325]]
step :  2000
cost :  0.419411
W1 : 
 [[ 0.28312019 -0.39471865 -1.11333168 -2.28917933  0.45707017  0.49649289
   2.41550422 -0.14644633  3.00679588  1.1011939 ]
 [ 0.87833893  1.63204575  0.25580126 -2.4157331   0.28779396 -1.50417733
  -1.20941222  1.3746171   3.04136539  0.7189784 ]]
W2 : 
 [[-0.66880953]
 [-1.45461786]
 [ 1.20022202]
 [-2.24934864]
 [-0.7094807 ]
 [ 1.337111  ]
 [-2.02765512]
 [-0.97450948]
 [ 3.25888014]
 [ 0.31411639]]
step :  4000
cost :  0.0603702
W1 : 
 [[ 0.53501052 -1.32142425 -2.48544598 -3.45974398  0.81918103  1.62729454

In [43]:
# Test NN Model
corrent_prediction = tf.equal(tf.floor(hypo + 0.5), Y)
accuracy = tf.reduce_mean(tf.cast(corrent_prediction, 'float'))

In [44]:
# Check Accuracy
print("hypo : \n", sess.run(hypo, feed_dict={X:x_data, Y:y_data}))
print("floor : \n", sess.run(tf.floor(hypo+0.5), feed_dict={X:x_data, Y:y_data}))
print("corrent_prediction : \n", sess.run(corrent_prediction, feed_dict={X:x_data, Y:y_data}))
print("accuracy : \n", sess.run(accuracy, feed_dict={X:x_data, Y:y_data}))

hypo : 
 [[ 0.0037942 ]
 [ 0.99040473]
 [ 0.99099052]
 [ 0.01305356]]
floor : 
 [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]]
corrent_prediction : 
 [[ True]
 [ True]
 [ True]
 [ True]]
accuracy : 
 1.0


In [29]:
sess.close()

## 결론 : NN Model 로 정확도 1.0 을 얻어냈다!

## "Wide" Neural Network for XOR problem

In [30]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [31]:
# Wide Neural Network 는 레이어를 구성할때, 더 많은 뉴런들로 구성하는 것을 말합니다.
W1 = tf.Variable(tf.random_uniform( [2,10], -1.0, 1.0))
W2 = tf.Variable(tf.random_uniform( [10,1], -1.0, 1.0))

In [32]:
b1 = tf.Variable(tf.zeros([10]), name="Bias1")
b2 = tf.Variable(tf.zeros([1]), name="Bias2")

In [33]:
# Define Hypo Fuction
L2 =  tf.sigmoid(tf.matmul(X,W1)+b1)
hypo = tf.sigmoid( tf.matmul(L2,W2) + b2)

In [34]:
# Define Cost function 
cost = -tf.reduce_mean( Y*tf.log(hypo)+(1-Y)* tf.log(1.-hypo))

In [35]:
# Minimize cost.
alpha = tf.Variable(0.1)
optimizer = tf.train.GradientDescentOptimizer(alpha)
train = optimizer.minimize(cost)

In [36]:
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

In [37]:
for step in range(10001):
    sess.run(train, feed_dict={X:x_data, Y:y_data})
    if step % 2000 == 0:
        print("step : ", step)
        print("cost : ", sess.run(cost, feed_dict={X:x_data, Y:y_data}))
        print("W1 : \n", sess.run(W1))
        print("W2 : \n", sess.run(W2))
        print("="*60)

step :  0
cost :  0.917693
W1 : 
 [[-0.22265865  0.05458229 -0.11917987 -0.48066619 -0.55019432  0.58600932
  -0.58177257  0.68136412  0.4146634   0.81032532]
 [ 0.0077646  -0.86452949  0.20272186  0.11087734 -0.05062024 -0.30531913
   0.10912818  0.7315492  -0.45006469  0.85396051]]
W2 : 
 [[ 0.08631957]
 [ 0.09290914]
 [-0.40912399]
 [ 0.05123728]
 [ 0.48107508]
 [-0.77188593]
 [ 0.62754953]
 [ 0.86122894]
 [ 0.92806929]
 [ 0.67687917]]
step :  2000
cost :  0.339809
W1 : 
 [[-0.00403753  0.04456377 -0.13380773 -0.23492903 -0.69496322  3.05959845
  -0.91268748  2.73229694  1.25980496  2.98526096]
 [ 0.35557896 -1.01806474  1.29845202  0.46704116 -0.16122191 -1.85186052
  -0.02098741  2.6164813  -2.4075861   2.86862612]]
W2 : 
 [[-0.55768949]
 [ 0.47413841]
 [-1.73288274]
 [-0.53204358]
 [ 0.37765852]
 [-3.29862022]
 [ 0.56611151]
 [ 2.37675905]
 [ 2.2396009 ]
 [ 2.72405124]]
step :  4000
cost :  0.0481751
W1 : 
 [[ 0.27473295 -0.00993435 -0.6543768   0.02882234 -0.92435902  5.264575
 

In [38]:
# Test wide NN Model
correct_prediction = tf.equal(tf.floor(hypo + 0.5), Y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [39]:
# Check Accuracy
print("hypo : \n", sess.run(hypo, feed_dict={X:x_data, Y:y_data}))
print("floor : \n", sess.run(tf.floor(hypo+0.5), feed_dict={X:x_data, Y:y_data}))
print("corrent_prediction : \n", sess.run(correct_prediction, feed_dict={X:x_data, Y:y_data}))
print("accuracy : \n", sess.run(accuracy, feed_dict={X:x_data, Y:y_data}))

hypo : 
 [[ 0.00416614]
 [ 0.98976058]
 [ 0.99152911]
 [ 0.01407304]]
floor : 
 [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]]
corrent_prediction : 
 [[ True]
 [ True]
 [ True]
 [ True]]
accuracy : 
 1.0


In [45]:
sess.close()

## 결론 : wide 하게 Neural 을 만들면서 cost 가 엄청나게 줄어든 모습을 확인할 수 있습니다.